# Niveau de bruit des établissements dans le référentiel des structures de HAL

objectif : produire un tableau avec les colonnes suivantes : nom de portail, nb structure, % de bruit

1. obtenir un tableau de tous les portails


2. réduire aux portails institutionnels


3. trouver les structId des portails identifiés
    - explorer l'API AureHal Structure pour avoir l'id des établissements à partir d'une requête par nom
    - créer une fonction qui donne à partir du nom de l'établissement leur id
    - ajouter au tableau des établissements une colonne avec leur id


4. pour chaque portail identifié extraire nb structure valide, non valide, % bruit 
    - créer une fonction qui donne : nb structure valide et non valide à partir d'un structId
    - ajouter les champs nb_struct_valid et nb_struct_no_valid au tableau des établissements


5. calculer le pourcetange de bruit et l'intégrer au tableau


6. retirer les etab où nous n'avons pas de structure valide, trier par noise en décroissant

In [1]:
import pandas as pd, json, requests

## 1. Obtenir la liste des portails HAL

charger la liste des portails  https://api.archives-ouvertes.fr/docs/ref/resource/instance

In [2]:
# récupérer les infos de tous les portails
raw = requests.get("https://api.archives-ouvertes.fr/ref/instance").json()

In [3]:
# les placer dans une dataframe
portails = pd.DataFrame(raw['response']['docs'])
portails.sample(3)

,id,code,name,url
89,92,afrique,Agence universitaire de la Francophonie,https://hal-auf.archives-ouvertes.fr
156,7264,univ-angers,Université d'Angers,https://hal.univ-angers.fr
69,76,espci,ESPCI Paris,https://hal-espci.archives-ouvertes.fr


In [4]:
print(f"nb de portails {len(portails)}")

nb de portails 186


## 2. Réduire aux protails institutionnels

étape semi automatique qui se fait cycliquement en constatant ce qui est pris ou non par le filtre

In [5]:
# une option pour afficher toutes les rows de la dataframe
pd.set_option('display.max_rows', None) 

In [6]:
# filtrer sur les portails des institutions
filtre_name = portails["name"].str.contains(
    "univ|ecole|école|inst|inria|inserm|cea|supelec|agro|cnam|enssib|ENS|CIrad|Observatoire|\
    |INSA|MNHN|inshea|lyon|MINES|HEC|rouen|Sciences Po|BRGM|ESSEC|ESPCI|Chimie ParisTech|nationale de France|Audencia|CSTB|\
    |onera|PSE|Rennes Business School|CentraleSupélec|NEOMA|EDF|IMT Atlantique|Télécom Paris|\
    |Centre national de la recherche scientifique|CHU|Collège de France|Kedge|isara|Météo",
case = False, regex=True)

portail_etab = portails[filtre_name].sort_values(by = "name")

print(f" nb de portails institutionnels repérés {len(portail_etab)}")

 nb de portails institutionnels repérés 156


In [7]:
# retirer les faux positifs récupérés par le filtre : afrique, dumas, hceres, jeannicod, riip
faux_positif = portail_etab[ portail_etab["code"].isin(["afrique", "dumas", "hceres", "jeannicod", "riip"])].index
portail_etab.drop(faux_positif, inplace = True)
print(f" nb de portails institutionnels repérés {len(portail_etab)}")

 nb de portails institutionnels repérés 151


In [8]:
## verifier la selection effectuée

# ce qui n'est pas retenu
#portails[ ~portails["id"].isin(portail_etab["id"].tolist())]

# ce qui est retenu
portail_etab


,id,code,name,url
61,67,agroparistech,AgroParisTech,https://hal-agroparistech.archives-ouvertes.fr
140,5852,agrosup-dijon,AgroSup Dijon,https://hal-agrosup-dijon.archives-ouvertes.fr
83,90,agrocampus-ouest,Agrocampus Ouest,https://hal-agrocampus-ouest.archives-ouvertes.fr
101,110,amu,Aix-Marseille Université,https://hal-amu.archives-ouvertes.fr
86,95,audencia,Audencia Business School,https://hal-audencia.archives-ouvertes.fr
58,65,brgm,BRGM - Bureau de Recherches Géologiques et Min...,https://hal-brgm.archives-ouvertes.fr
85,94,bnf,Bibliothèque nationale de France,https://hal-bnf.archives-ouvertes.fr
23,26,cea,CEA - Commissariat à l’énergie atomique,https://hal-cea.archives-ouvertes.fr
164,7636,chu-clermontferrand,CHU de Clermont-Ferrand,https://hal-chu-clermontferrand.archives-ouver...
87,96,cstb,CSTB - Centre Scientifique et Technique du Bât...,https://hal-cstb.archives-ouvertes.fr


## 3. Trouver les `structId` des portails identifiés

mémo documentation référentiel structure https://api.archives-ouvertes.fr/docs/ref/?resource=structure&schema=fields#fields

In [9]:
def find_portail_structId(name) : 
    """
    a partir du nom du portail retrouver le structId de l'établissement
    ne pas recherche par sigle car ça engendre bcp de faux positif (eg CEA)
    si aucun sutrcutId trouvé impression "pb" suivi du nom
    """
    # utiliser name_t car insensible à la casse et accents
    # memo : https://api.archives-ouvertes.fr/ref/structure/?fq=type_s:institution&q=(name_t:Institut Catholique de Paris)
    # certains noms contiennent le sigle de l'établissement avant - (ICP - Institut Catholique de Paris)
    
    pre_url = "https://api.archives-ouvertes.fr/ref/structure/?fq=type_s:institution&fq=valid_s:VALID"
        
    req = requests.get( f"{pre_url}&fq=name_t:\"{name}\"" )
    res = req.json()
    try : 
        # retourner le docid et le label
        return [ res["response"]["docs"][0]["docid"], res["response"]["docs"][0]["label_s"]]
    except : 
        print("pb", name)

In [10]:
# tester la fonction
## non optimal on releve des pb, eg "Université de Paris", rattrapé à la main par la suite
# pb aussi pour HEC qui va d'abord récupérer le HEC de montréal
# pb sur les old new eg avec l'univ de Tours
find_portail_structId("Commissariat à l’énergie atomique")

[300016,
 "Commissariat à l'énergie atomique et aux énergies alternatives [CEA]"]

In [11]:
# appliquer la fonction sur tous les établissements
portail_etab[["structId", "structId_name"]] = portail_etab.apply(
    lambda row : find_portail_structId(row["name"]), axis = 1, result_type="expand")

pb AgroSup Dijon
pb Agrocampus Ouest
pb BRGM - Bureau de Recherches Géologiques et Minières
pb CEA - Commissariat à l’énergie atomique
pb CHU de Clermont-Ferrand
pb CSTB - Centre Scientifique et Technique du Bâtiment
pb Cirad
pb Cnam - Conservatoire national des arts et métiers
pb EM-LYON Business School
pb ENAC - Ecole Nationale de l'Aviation Civile
pb ENS de Lyon
pb ENSSIB
pb ENSTA Bretagne
pb ENSTA Paris
pb EPHE - École pratique des hautes études
pb ESPCI Paris
pb ESSEC BusinessSchool
pb Ecole Centrale Paris
pb Groupe des Écoles Nationales d'Économie et Statistiques
pb Hal Lumière Lyon2
pb ICP - Institut Catholique de Paris
pb IN2P3 - Institut national de physique nucléaire et de physique des particules
pb INERIS - Institut National de l’EnviRonnement Industriel et des RisqueS
pb INSA Lyon
pb INSA Rennes
pb INSA-Toulouse
pb INSHEA
pb INSU - Institut national des sciences de l'Univers
pb IRD - Institut de recherche pour le développement
pb IRSN - Institut de Radioprotection et de Sûr

In [12]:
## extraire le tableau pour analyser le matching automatique
portail_etab.sort_values("structId", inplace = True)
portail_etab.to_csv("verif-portail-list-match-structId.csv", index = False)

In [13]:
# calculer taux de détection de structId de porail
print( f"Avec la fonction find_portail_structId\nstructId trouvés pour { round(len( portail_etab[ portail_etab['structId'].notna()] ) / len(portail_etab) * 100, 1) } % des étab.") 

Avec la fonction find_portail_structId
structId trouvés pour 51.7 % des étab.


In [14]:
## charger le matching manuel
matching_manu = json.load(open("../data/2022-06-manually-match-portail-to-structId.json", encoding='utf-8'))

In [15]:
# le matching manuel contient deux listes dont les clés sont les suivantes
# portail_full_name_match_error & portail_full_name_no_match

portail2structId = matching_manu["portail_full_name_match_error"] 
portail2structId.update(matching_manu["portail_full_name_no_match"])
print("nombre de mathcing effectués manuellement", len(portail2structId))

nombre de mathcing effectués manuellement 94


In [16]:
# ajouter/rectifier les données avec le matching manuel
# si le nom de l'étab est dans le json alors écraser le structId sinon le conserver

portail_etab["structId"] = portail_etab.apply(
lambda x : portail2structId[x["name"]] if x["name"] in portail2structId.keys() else x["structId"], axis = 1)

In [17]:
# calculer taux de détection de structId de porail
print( f"Apres matching manuel\nstructId trouvés pour { round(len( portail_etab[ portail_etab['structId'].notna()] ) *100 / len(portail_etab), 0) }  % des étab.") 

Apres matching manuel
structId trouvés pour 100.0  % des étab.


In [18]:
print(f"nombre etablissement sans structId restant {len( portail_etab[portail_etab['structId'].isna()]) }", f"\n\nnb etablissement total {len(portail_etab)}")

nombre etablissement sans structId restant 0 

nb etablissement total 151


## 4. Obtenir le nombre de structure par établissement

In [19]:
def get_struct_nb(structId, valid):
    """
    nombre de sous structures attachées à la structure parente
    structId : l'id de la struct parente
    valid si besoin  de filtrer par état (* pour tout inclure)
    """
    # si l'établissement n'a pas de structId alors on ne le traite pas
    if not pd.isna(structId) :  
        url = "https://api.archives-ouvertes.fr/ref/structure/?"
        req = requests.get(url+ f"&q=parentDocid_i:{int(structId)}&fq=valid_s:{valid}&rows=0")
        req = req.json()
        return req["response"]["numFound"]    

In [20]:
# test de la foncion
get_struct_nb(172265,"VALID")

29

* Ajouter nb structure total de l'établissement et nb structure incoming

In [21]:
portail_etab["nb_struct"] = portail_etab.apply(lambda row : get_struct_nb(row['structId'], "*"), axis = 1)
portail_etab.sample(2)

,id,code,name,url,structId,structId_name,nb_struct
92,99,icp,ICP - Institut Catholique de Paris,https://hal-icp.archives-ouvertes.fr,301937.0,None,8
30,33,paris1,Université Paris 1 Panthéon-Sorbonne,https://hal-paris1.archives-ouvertes.fr,7550.0,Université Paris 1 Panthéon-Sorbonne [UP1],228


In [22]:
portail_etab["nb_struct_incoming"] = portail_etab.apply(lambda row : get_struct_nb(row['structId'], "INCOMING"), axis = 1)

In [23]:
portail_etab.sample(2)

,id,code,name,url,structId,structId_name,nb_struct,nb_struct_incoming
130,5385,ehesp,Ecole des hautes études en santé publique,https://hal.ehesp.fr,301986.0,École des Hautes Études en Santé Publique [EHE...,36,14
167,7802,ip-paris,Institut Polytechnique de Paris,https://hal-ip-paris.archives-ouvertes.fr,563936.0,None,0,0


* Calculer le pourcentage de bruit et l'intégrer au tableau

In [25]:
portail_etab["noise"] = portail_etab["nb_struct_incoming"] / portail_etab["nb_struct"]
portail_etab.sample(1)

,id,code,name,url,structId,structId_name,nb_struct,nb_struct_incoming,noise
127,5241,univ-rennes2,Université Rennes 2,https://hal.univ-rennes2.fr,406201.0,None,132,14,0.106061


In [26]:
print("nb d'établissement", len(portail_etab))

nb d'établissement 151


* Nettoyer le tableau et afficher le classement

In [27]:
# retrait des lignes sans données pour nb_struct et noise
portail_noise = portail_etab[ 
    (portail_etab["nb_struct"].notna()) & 
    (portail_etab["noise"].notna()) 
]
print("Après nettoyage nb d'établissement", len(portail_etab))

Après nettoyage nb d'établissement 151


In [28]:
# imprimer les résultats pour les étab où il y au moins une structure
portail_etab.sort_values("noise", ascending = False, inplace = True)
portail_etab[ portail_etab["nb_struct"] > 0 ]

,id,code,name,url,structId,structId_name,nb_struct,nb_struct_incoming,noise
26,29,irsn,IRSN - Institut de Radioprotection et de Sûret...,https://hal-irsn.archives-ouvertes.fr,300040.0,None,280,249,0.889286
64,71,hcl,Hospices Civils de Lyon,https://hal-hcl.archives-ouvertes.fr,300074.0,"Hospices Civils de Lyon, Departement de Neurol...",478,421,0.880753
22,25,ineris,INERIS - Institut National de l’EnviRonnement ...,https://hal-ineris.archives-ouvertes.fr,3186.0,None,14,12,0.857143
139,5836,edf,EDF,https://hal-edf.archives-ouvertes.fr,302313.0,EDF [EDF],74,56,0.756757
149,6616,em-lyon,EM-LYON Business School,https://hal.em-lyon.com,301362.0,None,4,3,0.750000
92,99,icp,ICP - Institut Catholique de Paris,https://hal-icp.archives-ouvertes.fr,301937.0,None,8,6,0.750000
162,7468,enva,École Nationale vétérinaire d'Alfort,https://hal-enva.archives-ouvertes.fr,34256.0,None,104,76,0.730769
97,104,pse,PSE - Paris School of Economics,https://hal-pse.archives-ouvertes.fr,301309.0,None,6,4,0.666667
67,74,essec,ESSEC BusinessSchool,https://hal-essec.archives-ouvertes.fr,301020.0,None,24,16,0.666667
81,88,univ-corse,Université de Corse Pascal Paoli,https://hal-univ-corse.archives-ouvertes.fr,508779.0,None,3,2,0.666667


* extraire les données 

In [29]:
# preciser la date dans le nom du fichier extrait
csv_file_name = "2022-06-aurehal-struct-noise-level-per-institution"
portail_noise.to_csv(f"../data/{csv_file_name}.csv", index = False)

In [30]:
#imprimer les stats sur l'indicateur
portail_noise["noise"].describe()

count    146.000000
mean       0.213932
std        0.232897
min        0.000000
25%        0.016439
50%        0.132240
75%        0.343786
max        0.889286
Name: noise, dtype: float64

---
Allez plus loin 
    

* récursif : comptes les "incoming" au sein des structures filles de la struct de l'établissement #attention couteut en temps de requête